In [1]:
import re
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns
import string
import nltk
import warnings 
import tweepy
warnings.filterwarnings("ignore", category=DeprecationWarning)

%matplotlib inline

In [86]:
keys = pd.read_csv('.keys.csv')

In [87]:
consumer_key = keys['consumer_key']
consumer_secret = keys['consumer_secret']
access_token = keys['access_token']
access_token_secret = keys['access_token_secret']
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

KeyError: 'access_token_secret'

In [40]:
place = api.geo_search(query="Peru", granularity="country")

In [42]:
place_id = place[0].id
print(place_id)

2dfa9ecb0179a4e4


In [78]:
tweets = []
for tweet in tweepy.Cursor(api.search, q='ciencia OR datos OR science OR data place:%s' % place_id).items(200):
    tweets.append(tweet)
print(len(tweets))

130


In [79]:
tweets_df = pd.DataFrame(vars(tweets[i]) for i in range(len(tweets)))
tweets_df.head()

,_api,_json,author,contributors,coordinates,created_at,entities,extended_entities,favorite_count,favorited,...,quoted_status,quoted_status_id,quoted_status_id_str,retweet_count,retweeted,source,source_url,text,truncated,user
0,<tweepy.api.API object at 0x7f8ba4ed48d0>,{'created_at': 'Sun Mar 03 21:54:14 +0000 2019...,User(_api=<tweepy.api.API object at 0x7f8ba4ed...,None,"{'type': 'Point', 'coordinates': [-71.5369, -1...",2019-03-03 21:54:14,"{'hashtags': [{'text': 'Wabis', 'indices': [9,...",NaN,0,False,...,NaN,NaN,NaN,0,False,Instagram,http://instagram.com,Qué tal? #Wabis\n\nLes invitamos a que nos sig...,True,User(_api=<tweepy.api.API object at 0x7f8ba4ed...
1,<tweepy.api.API object at 0x7f8ba4ed48d0>,{'created_at': 'Sun Mar 03 21:42:11 +0000 2019...,User(_api=<tweepy.api.API object at 0x7f8ba4ed...,None,None,2019-03-03 21:42:11,"{'hashtags': [], 'symbols': [], 'user_mentions...",NaN,5,False,...,Status(_api=<tweepy.api.API object at 0x7f8ba4...,1.102322e+18,1102322223719768064,0,False,Twitter for Android,http://twitter.com/download/android,1. Que interesa que sea de provincias? 2. Eso ...,True,User(_api=<tweepy.api.API object at 0x7f8ba4ed...
2,<tweepy.api.API object at 0x7f8ba4ed48d0>,{'created_at': 'Sun Mar 03 20:28:45 +0000 2019...,User(_api=<tweepy.api.API object at 0x7f8ba4ed...,None,None,2019-03-03 20:28:45,"{'hashtags': [], 'symbols': [], 'user_mentions...",NaN,0,False,...,NaN,NaN,NaN,0,False,Twitter Web Client,http://twitter.com,@AndresMunday @angabantxo @DrBioblogo @GrntyTH...,True,User(_api=<tweepy.api.API object at 0x7f8ba4ed...
3,<tweepy.api.API object at 0x7f8ba4ed48d0>,{'created_at': 'Sun Mar 03 19:37:40 +0000 2019...,User(_api=<tweepy.api.API object at 0x7f8ba4ed...,None,None,2019-03-03 19:37:40,"{'hashtags': [], 'symbols': [], 'user_mentions...",NaN,1,False,...,Status(_api=<tweepy.api.API object at 0x7f8ba4...,1.102247e+18,1102247246425440261,0,False,Twitter for iPhone,http://twitter.com/download/iphone,Ahí unos datos para su conocimiento @Libertado...,True,User(_api=<tweepy.api.API object at 0x7f8ba4ed...
4,<tweepy.api.API object at 0x7f8ba4ed48d0>,{'created_at': 'Sun Mar 03 19:27:58 +0000 2019...,User(_api=<tweepy.api.API object at 0x7f8ba4ed...,None,None,2019-03-03 19:27:58,"{'hashtags': [], 'symbols': [], 'user_mentions...",NaN,0,False,...,NaN,NaN,NaN,0,False,Twitter for Android,http://twitter.com/download/android,"Cuando Emmanuel Carrère leyó a Philip K. Dick,...",True,User(_api=<tweepy.api.API object at 0x7f8ba4ed...


In [80]:
tweet_atts = [
'text', 'created_at', 'favorite_count',
'lang', 'retweet_count', 'source',
'in_reply_to_user_id_str', 'retweeted',
'id'
]

# subset dataframe
tweets_df = tweets_df[tweet_atts]

In [81]:
tweets_df.head()

,text,created_at,favorite_count,lang,retweet_count,source,in_reply_to_user_id_str,retweeted,id
0,Qué tal? #Wabis\n\nLes invitamos a que nos sig...,2019-03-03 21:54:14,0,es,0,Instagram,None,False,1102326342673096705
1,1. Que interesa que sea de provincias? 2. Eso ...,2019-03-03 21:42:11,5,es,0,Twitter for Android,None,False,1102323306219352064
2,@AndresMunday @angabantxo @DrBioblogo @GrntyTH...,2019-03-03 20:28:45,0,es,0,Twitter Web Client,1061636832918679553,False,1102304829152604160
3,Ahí unos datos para su conocimiento @Libertado...,2019-03-03 19:37:40,1,es,0,Twitter for iPhone,None,False,1102291973241794560
4,"Cuando Emmanuel Carrère leyó a Philip K. Dick,...",2019-03-03 19:27:58,0,es,0,Twitter for Android,None,False,1102289531347394560


In [83]:
tweets_df.to_csv('../data/raw/tweets.csv')